# Tool Use 도구 사용의 구조

- 도구 설정하기
- 도구 사용 워크플로우 실행하기

## 도구 설정하기

먼저, OpenAI 파이썬 라이브러리를 설치하고 클라이언트를 설정하자.

In [ ]:
%pip install -q openai python-dotenv

In [19]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv  

!echo "OPENAI_API_KEY=<Open AI 발급한 Key를 입력하세요." >> .env #최초만 설정하여 실행
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI()

### 1단계: 도구 생성하기

In [20]:
def daily_sales_report(day: str) -> str:
    """
    주어진 날짜의 판매 보고서를 검색하는 함수다.
    """
    # 일일 판매 보고서를 포함하는 모의 데이터베이스
    sales_database = {
        '2025-08-28': {'total_sales_amount': 1000000, 'total_units_sold': 100},
        '2025-08-29': {'total_sales_amount': 15000000, 'total_units_sold': 150},
        '2025-08-30': {'total_sales_amount': 8000000, 'total_units_sold': 80}
    }
    
    report = sales_database.get(day, {})
    
    if report:
        result = {
            'date': day,
            'summary': f"총 판매 금액: {report['total_sales_amount']}, 총 판매 수량: {report['total_units_sold']}"
        }
    else:
        result = {'date': day, 'summary': '해당 날짜에 대한 판매 데이터가 없습니다.'}
        
    return json.dumps(result, ensure_ascii=False)

# 도구 이름과 실제 함수를 매핑한다.
available_tools = {
    "daily_sales_report": daily_sales_report
}

### 2단계: 도구 스키마 정의하기

In [21]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "daily_sales_report",
            "description": "데이터베이스에 연결하여 특정 날짜의 전체 판매량 및 판매 정보를 검색한다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "day": {
                        "type": "string",
                        "description": "YYYY-MM-DD 형식으로 된, 판매 데이터를 검색할 날짜.",
                    }
                },
                "required": ["day"]
            }
        }
    }
]

### 3단계: 사용자 지정 시스템 프롬프트 생성하기

In [22]:
system_prompt = """당신은 사용자의 질문과 요청에 대화형으로 답변하는 AI 어시스턴트입니다. 
다양한 주제에 대한 광범위한 요청을 받게 될 것입니다. 
답변을 조사하는 데 도움이 되는 다양한 검색 엔진이나 유사한 도구를 갖추고 있습니다. 
사용자의 요구를 최선을 다해 지원하는 데 집중해야 합니다.
사용자가 다른 스타일의 답변을 요청하지 않는 한, 완전한 문장으로 올바른 문법과 철자법을 사용하여 답변해야 합니다.
"""

## 도구 워크플로우 실행하기
가장 기본적으로, 이 네 구성 요소는 네 단계의 워크플로우를 통해 상호 작용한다:

- **1단계: 사용자 메시지 받기**. LLM이 (애플리케이션을 통해) 사용자 메시지를 받는다.
- **2단계: 도구 호출 생성하기**. LLM이 어떤 도구를 호출할지(필요하다면) 결정하고 도구 호출을 생성한다.
- **3단계: 도구 결과 얻기**. 애플리케이션이 도구를 실행하고, 그 결과가 LLM에 전송된다.
- **4단계: 응답 및 인용 생성하기**. LLM이 사용자에게 응답과 인용을 생성하여 반환한다.

### 1단계: 사용자 메시지 받기

In [23]:
message = "2025년 8월 29일의 판매 요약을 제공해 줄 수 있나요?"

### 2단계: 도구 호출 생성하기

In [24]:
MODEL = "gpt-4o-mini"

# 대화 기록을 관리할 메시지 리스트
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": message}
]

# 1차 API 호출: 모델이 도구를 사용할지 결정한다.
response = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

response_message = response.choices[0].message
tool_calls = response_message.tool_calls

if tool_calls:
    print("도구 호출이 생성되었습니다:")
    for i, t in enumerate(tool_calls):
        print(f"#{i+1}\n도구: {t.function.name}\n매개변수: {t.function.arguments}\n")
else:
    print("모델이 도구를 호출하지 않고 바로 답변했습니다:")
    print(response_message.content)

도구 호출이 생성되었습니다:
#1
도구: daily_sales_report
매개변수: {"day":"2025-08-29"}



### 3단계: 도구 결과 얻기

In [25]:
# 모델의 도구 호출 결정을 메시지 기록에 추가한다.
messages.append(response_message)

# 도구 호출을 실행하고 결과를 수집한다.
for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_tools[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(**function_args)
    
    # 도구 실행 결과를 메시지 기록에 추가한다.
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )

print("도구 결과:")
print(messages[-1]) # 마지막에 추가된 tool 메시지 출력

도구 결과:
{'tool_call_id': 'call_smxh4JgIM5rlNFLPhGPgsRMV', 'role': 'tool', 'name': 'daily_sales_report', 'content': '{"date": "2025-08-29", "summary": "총 판매 금액: 15000000, 총 판매 수량: 150"}'}


### 4단계: 응답 생성하기

In [26]:
# 2차 API 호출: 도구 결과를 바탕으로 최종 답변을 생성한다.
final_response = client.chat.completions.create(
    model=MODEL,
    messages=messages
)

print("최종 응답:")
print(final_response.choices[0].message.content)
print("="*50)

최종 응답:
2025년 8월 29일의 판매 요약은 다음과 같습니다:

- 총 판매 금액: 15,000,000원
- 총 판매 수량: 150개

더 궁금한 점이 있다면 말씀해 주세요!


LLM의 최종 응답은 다음과 같으며, 사용자의 질문(총 판매 금액: 15,000,000, 총 판매 수량: 150)에 정확하게 답변한다.

### 응답 스트리밍하기

In [27]:
# 스트리밍으로 최종 응답 생성
stream = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    stream=True,
)

print("최종 응답 (스트리밍):")
for chunk in stream:
    content = chunk.choices[0].delta.content
    if content:
        print(content, end="", flush=True)
print()  # 줄바꿈

최종 응답 (스트리밍):
2025년 8월 29일의 판매 요약은 다음과 같습니다: 

- 총 판매 금액: 15,000,000원
- 총 판매 수량: 150개
